In [ ]:
#from skimage.io import imread
#import pyclesperanto_prototype as cle  # version 0.19.4
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from skimage.io import imread 

**Preliminary question to find the files and the status of the analysis** 

In [ ]:
first_time = input("Is this the first strain?(yes/no)")
first_field = input("Is this the first field of view for this strain?(yes/no)")
folder = input("Paste the URL of where you  want to save/upload data")
folder_path = Path(folder)
filename = "intensity.csv"
full_path = folder_path.joinpath(filename)
strain = input("Enter the strain name: ")

intensity_dict = []
if first_time == "no":
    intensity_df = pd.read_csv(full_path)
    # convert the dataframe to a dictionary
    intensity_dict = intensity_df.to_dict('records')

n_worm = 0    
if first_field == "no":
    for data_dict in intensity_dict:
        if str(data_dict['strain']) == str(strain) and data_dict['worm'] > n_worm:
            n_worm = data_dict['worm']

print("number of worms already analysed")
n_worm            

**Getting into the folder where imagese are**

In [ ]:
directory  = r"S:\microscopy\HSP\24.05.16 srx-9\3472\analysis"
directory_path = Path(directory)
strain_file= input("Enter the field of view name: ")
strain_file_format = strain_file+ '.tif'
# Import functions from the custom module binding_box_finder
from binding_box_finder_v14 import blob_maker_DIC, binding_box_find_and_display
images  = imread(os.path.join(directory_path, strain_file_format))
images = images[:,:,:]
w, h = images[0].shape

w_2_3 = 2 * int(w/3)
w_3 = int(w/3)
h_2_3 = 2 * int(h/3)
h_3 = int(h/3)

coord_list = [(0, 0, w_3, h_3), (0, h_3, w_3, h_3), (0, h_2_3, w_3, h_3), 
             (w_3, 0, w_3, h_3), (w_3, h_3, w_3, h_3), (w_3, h_2_3, w_3, h_3),
             (w_2_3, 0, w_3, h_3), (w_2_3, h_3, w_3, h_3), (w_2_3, h_2_3, w_3, h_3)]

coord_list = [
             (w_3, h_3, w_3, h_3), 
             (w_2_3, 0, w_3, h_3), (w_2_3, h_3, w_3, h_3), (w_2_3, h_2_3, w_3, h_3)]



In [ ]:
#### Import the draw_boxes_from_coordinates function from the custom module binding_box_finder
from binding_box_finder_v14 import draw_boxes_from_coordinates as dbfc

# Specify the filename for the image with drawn bounding boxes
filename_boxes = os.path.join(directory_path, 'boxes_' + strain_file + '.jpeg')


# Use the draw_boxes_from_coordinates function to draw bounding boxes on the first image in images_gcamp
# The function likely takes coord_list, the image, and the filename as parameters
_ = dbfc(coord_list, np.max(images, axis=0), filename_boxes, n_worm+1)


**Single worm analysis, getting mean intensity**

In [ ]:
from skimage import filters

# Iterate through each day's image in the dataset
for day, img in enumerate(images):
    
    if first_field == "no":
        n_worm = data_dict['worm']
    else: 
        n_worm = 0
    # Iterate through the list of worm coordinates
    for xy in coord_list:
        n_worm += 1 
        x,y, w, h = xy
        # Extract the worm region from the image
        worm  = img[y:(y+h), x:(x+w)]
        # Apply Otsu's thresholding for binarization
        worm_trash = filters.threshold_otsu(worm)
        # Feature extraction: Identify pixels above the threshold
        worm_over_trash = worm > worm_trash
        # Compute mean intensity of the worm region above the threshold
        worm_itensity = np.nanmean(worm[worm_over_trash])
        
        # Create a dictionary to store extracted data for the worm
        worm_data = {"day": int(day+1), "strain":str(strain), "worm": int(n_worm), "mean_intensity": float(worm_itensity)}
        
        # Append the worm data dictionary to the list
        intensity_dict.append(worm_data)
      
        
# create the dataframe from the list of data
intensity_df = pd.DataFrame(intensity_dict)
# save the dataframe as csv file without index
intensity_df.to_csv(full_path, index=False) #False exclude index column

intensity_df


In [ ]:


folder_path2 = Path(r'S:\microscopy\HSP\24.05.28 srx-9\3472\analysis')
plot_data = pd.read_csv(folder_path2.joinpath("intensity_norm_rep1.csv"))


normalized_df = plot_data
plot_data

In [ ]:
# a list of tuples where each tuple is (strain, worm)
worms_to_remove = [(3470, 3), (3470, 6), (3470, 12), (3470, 14), (3470, 16), (3470, 20), (3470, 23), (3470, 25), (3470, 31)]

# Filter the data
for strain, w in worms_to_remove:
    plot_data = plot_data[~((plot_data['strain'] == strain) & (plot_data['worm'] == w))]

normalized_df = plot_data
plot_data


In [ ]:

# Function to normalize intensity measurements relative to day 1
def normalize_group(group):
    # Check if there is a day 1 measurement
    if 1 in group['day'].values:
        day_1_mean_intensity = group.loc[group['day'] == 1, 'mean_intensity'].values[0]
        # Compute normalized intensity as a fraction of day 1's intensity
        group['normalized_intensity'] = (group['mean_intensity'] - day_1_mean_intensity) / day_1_mean_intensity
    else:
        group['normalized_intensity'] = None  # Assign None if day 1 data is missing
    return group

# Apply normalization for each worm within each strain and replicate group
normalized_df = plot_data.groupby(['strain', 'worm', 'replicate']).apply(normalize_group)



# Reset index to keep the dataframe clean
normalized_df = normalized_df.reset_index(drop=True)

In [ ]:
normalized_df.to_csv(folder_path2.joinpath('intensity_norm.csv'), index=False) #False exclude index column
normalized_df

In [ ]:
# Group by strain
grouped_norm = normalized_df.groupby(['strain', 'day'])
# Calculate the mean normalized intensity for each group
mean_by_group = grouped_norm['normalized_intensity'].mean()
# Calculate the standard deviation for each group
std_by_group = grouped_norm['normalized_intensity'].std()
# Count the number of worms for each group
count_by_group = grouped_norm['normalized_intensity'].count()
# Calculate the standard error for each group
standard_error_by_group = std_by_group / np.sqrt(count_by_group)

# Create a new DataFrame containing the strain, day, mean normalized intensity, and standard error
avg_norm_df = pd.DataFrame({
    'strain': mean_by_group.index.get_level_values('strain'),
    'day': mean_by_group.index.get_level_values('day'),
    'mean_normalized_intensity': mean_by_group.values,
    'standard_error': standard_error_by_group.values
})

avg_norm_df.to_csv(folder_path2.joinpath('intensity_avg_norm_error.csv'), index=False) 
avg_norm_df